In [ ]:
#| default_exp bridget


In [ ]:
#| export
from __future__ import annotations


# Bridget
> HTMX + FastHTML - Server for Jupyter Notebooks

`AnyWidget` widget standing in for an HTTP server with `HTMX` Ajax calls.

## Why Bridget?



Jupyter notebooks serve two primary functions:
1. **Exploratory/Educational**: Interactive computing and learning
2. **Development**: Literate programming and package creation (via nbdev)

While code creation is well-served by various notebook environments (Jupyter, VSCode, Marimo, etc.), the display and interaction capabilities are often limited by IPython's well-thought but basic and aging display system. Modern development requires:

- Rich data visualization
- Interactive system integration
- Better control over output rendering

### The Front-end Challenge

Notebooks already run in browsers (or browser-like environments), giving us access to powerful HTML/JavaScript capabilities. The front-end and kernel are inherently connected, suggesting we shouldn't need additional communication layers.

However, the notebook ecosystem is fragmented:
- Jupyter (nbclassic, Notebook, Lab): ipywidgets
- VSCode/Cursor: Extensions
- Marimo: Custom solutions
- Google Colab: Proprietary package

This fragmentation forces users needing better display solutions to deal with complex, environment-specific tooling.

### Enter FastHTML and HTMX

FastHTML provides an innovative approach to web apps, emphasizing HTML-first development through HTMX. While Jupyter support isn't its core focus, FastHTML itself is developed using nbdev in notebooks.

Current FastHTML notebook integration:
- Launches a separate Uvicorn server
- Connects HTMX via standard HTTP/AJAX
- Works across most notebook variants

This approach is general, clean and lightweight but involves spawning a full HTTP server with:
- Multi-user capabilities (unnecessary for notebooks)
- Async architecture (notebooks are sync)
- Complex lifecycle management
- Production-level features (overkill for notebook use)
- IPython Javascript display object in each cell to trigger HTMX.

### The Bridget Solution

Bridget proposes using a widget-based approach that:
1. **Simplifies**: Replaces HTTP server with direct widget communication
2. **Generalizes**: Works across notebook environments via AnyWidget
3. **Extends**: Enables creation of notebook-specific components
4. **Integrates**: Provides Python API for HTMX functionality

This proof-of-concept shows how we can maintain FastHTML's powerful features while better adapting to the notebook environment's unique characteristics and constraints.

:::{.callout-note}
Personal note: My interest here isn't web apps, but notebook development.  

I like ipywidgets, or at least their intentions. I've written several and used them in many personal projects. However, they're a challenging piece of software with complicated tooling. They inherit all the nightmarish complexity of the JavaScript ecosystem, where the tooling is more involved than the language itself. AnyWidget is a step in the right direction, liberating us Pythonistas from the JS ecosystem. But the notebook part, the Python part, remains an unsolved problem in my opinion.

Of all solutions I've explored for achieving full interactivity in notebooks, HTMX + FastHTML comes closest, feeling more natural and integrating better with the notebook environment.
:::

# Prologue

In [ ]:
#| export

import inspect
import time
from pathlib import Path
from types import MethodType
from typing import Any
from typing import Mapping
from typing import Protocol
from typing import TypeAlias

import anywidget
import fastcore.all as FC
import traitlets as T
from anyio import from_thread
from fastcore.xml import escape
from fastcore.xml import FT
from fastcore.xml import NotStr
from fastcore.xml import to_xml
from fasthtml.core import APIRouter
from fasthtml.core import Client
from fasthtml.core import FastHTML
from httpx import ASGITransport
from httpx import AsyncClient
from httpx import codes
from httpx import Request
from httpx import Response
from IPython.display import display
from IPython.display import HTML
from olio.common import update_


In [ ]:
#| export

from fasthtml.components import Div, Details, Summary, B, Pre, Span


In [ ]:
#| export

import bridget
from bridget.cell_display import DisplayId
from bridget.display_helpers import BasicLogger
from bridget.display_helpers import nb_app
from bridget.display_helpers import pretty_repr
from bridget.helpers import bridge_cfg
from bridget.helpers import id_gen
from bridget.htmx import swap
from bridget.htmx import SwapStyleT
from bridget.routing import add_routes
from bridget.routing import RouteProviderP
from bridget.widget_helpers import anysource
from bridget.widget_helpers import BlockingMixin


In [ ]:
import dataclasses
import json
import random
from contextlib import contextmanager
from inspect import Parameter
from typing import cast

import fasthtml.components as ft
from bridget.routing import RouteProvider
from fastcore.test import *
from fasthtml.basics import ft_html
from fasthtml.components import Button
from fasthtml.components import H1
from fasthtml.components import H2
from fasthtml.components import Input
from fasthtml.components import Label
from fasthtml.components import Li
from fasthtml.components import Main
from fasthtml.components import show
from fasthtml.components import Spanconsole
from fasthtml.components import Text
from fasthtml.components import Title
from fasthtml.components import Ul
from fasthtml.xtend import Style
from IPython.display import DisplayHandle
from IPython.display import Javascript
from IPython.display import Markdown
from olio.common import setup_console


In [ ]:
from bridget.routing import ar
from bridget.widget_helpers import cleanupwidgets


----


In [ ]:
#| exporti

DEBUG = True
new_id = id_gen()
IDISPLAY = display


In [ ]:
console, cprint = setup_console(140)


In [ ]:
# needed for vfile:
%load_ext anywidget


In [ ]:
%env ANYWIDGET_HMR=0

env: ANYWIDGET_HMR=0


----

In [ ]:
#| exporti

BUNDLE_PATH = Path() if __name__ == "__main__" else Path(inspect.getfile(bridget)).parent


In [ ]:
_EMPTY = Parameter.empty
_VOID = object()


In [ ]:
#| exporti

_n = '\n'

# Helpers
> Some convenience utils to work with FastHTML in Notebooks.

## Bridget scripts

`HTMX` and other useful JS libraries.

In [ ]:
#| export

class ScriptsDetails:
    def __init__(self, scs, title='Loaded scripts', open=True): 
        self.scs = scs; self.title = title; self.open = open
    def __ft__(self):
        return Details(open=self.open)(
            Summary(B(self.title)),
            Pre(NotStr('\n'.join(escape(to_xml(_, indent=False, do_escape=False).strip()) for _ in self.scs))),
        )

def _bridget_scripts_extra():
    from fasthtml.core import surrsrc, scopesrc
    return {'surreal': surrsrc, 'css_scope_inline': scopesrc}

def _bridget_scripts(htmx=True):
    from fasthtml.core import fhjsscr
    from fasthtml.xtend import Script
    htmxsrc = Script(src=f"https://unpkg.com/htmx.org@next/dist/htmx.{'' if DEBUG else 'min.'}js")()
    return update_({'htmx': htmxsrc } if htmx else {}, fasthtml_js=fhjsscr, **_bridget_scripts_extra())
def _load_scripts(scs):
    display(HTML(to_xml((*(scvals := [_ for _ in scs.values()]), ScriptsDetails(scvals)))))
def bridget_scripts(load=False, htmx=True):
    scs = _bridget_scripts(htmx)
    if load: _load_scripts(scs)
    return scs


In [ ]:
bridget_scripts(True);


In [ ]:
html = '''
<div>💩 👻 No style.</div>

<div>
    <style> /* Simple example. */
        me { margin: 20px; }
        me div { font-size: 5rem; }
    </style>
    <div>👻</div>
</div>
'''

display(HTML(html))

Bridget automatically loads required JavaScript libraries:

1. HTMX
2. FastHTML core scripts
3. Awesome gnat's Scope and Surreal scripts

These are loaded by default when creating a Bridget app. You can customize this by overriding defaults when creating the app (covered in app creation section)


## ClientP

In [ ]:
#| export

# if typing.TYPE_CHECKING:
class ClientP(Protocol):
    def get(self, url: str, **kwargs) -> Response: ...
    def post(self, url: str, **kwargs) -> Response: ...
    def delete(self, url: str, **kwargs) -> Response: ...
    def put(self, url: str, **kwargs) -> Response: ...
    def patch(self, url: str, **kwargs) -> Response: ...
    def options(self, url: str, **kwargs) -> Response: ...


# Bridge

In [ ]:
def get_app(): 
    app = FastHTML(default_hdrs=False, sess_cls=None)  # type: ignore
    app.user_middleware.clear()
    return app, AsyncClient(transport=ASGITransport(app), base_url='http://nb'), app.route

app, cli, rt = get_app()

@rt("/hi")
def get():
    return 'Hi there'


## Bridge utils

In [ ]:
#| export

def request2httpx_request(cli:AsyncClient, http_request: dict[str, Any]) -> Request:
    r = http_request
    return cli.build_request(r['method'], r['url'], 
        headers=r['headers'] if 'headers' in r else {}, 
        content=r['body'] if 'body' in r else None, timeout=None)


In [ ]:
#| export

class HasFT(Protocol): 
    def __ft__(self) -> Any: ...
class HasHTML(Protocol):
    def __html__(self) -> str: ...

Bridgeable: TypeAlias = str|Mapping|FT|HasFT|HasHTML


def request2response(cli:AsyncClient, http_request) -> Response:
    httpx_request = request2httpx_request(cli, http_request)
    with from_thread.start_blocking_portal() as portal: 
        response = portal.call(cli.send, httpx_request)
    return response


def httpx_response_to_json(response: Response) -> dict[str, Any]:
    hdrs = {**response.headers, 
        'last-modified': time.strftime('%a, %d %b %Y %H:%M:%S GMT', time.gmtime()),
        'cache-control': 'no-store, no-cache, must-revalidate',
    }
    data = response.content.decode()
    json_response = {
        "headers": hdrs,
        "status": response.status_code,
        "statusText": codes.get_reason_phrase(response.status_code),
        # "text": data,
        "data": data,
        "xml": None,
        'finalUrl': f"{response.request.url}",
    }
    return json_response


In [ ]:
@FC.patch
def _ipython_display_(self: Response):
    dhdl = DisplayId()
    dhdl.display(self.text)


## BridgeBase
> A mixin class that wraps FastHTML and Client functionality for displaying and mounting components.


In [ ]:
#| export

class BridgeBase:
    "A simple wrapper around `FastHTML` and `Client`."

    def setup(self, app: FastHTML):
        self.app = app
        cli:ClientP = Client(self.app, 'http://nb)')  # type: ignore
        self.cli = cli
        self._cli = AsyncClient(transport=ASGITransport(self.app), base_url='http://nb')#, headers={'hx-request': '1'})
        return self
    
    def __call__(self, rt:Bridgeable='', method='GET', req=None, **kwargs):
        "Display FastHTML components, routes or requests in notebook cells."
        if isinstance(rt, FT) or hasattr(rt, '__ft__'): cts = to_xml(rt)  # type: ignore
        elif hasattr(rt, '__html__'): cts = rt.__html__()  # type: ignore
        else:
            if isinstance(rt, Mapping):
                http_request = {**(req or {}), **rt}
                if 'method' not in http_request: http_request['method'] = method
            else: http_request = {'headers': {'hx-request': '1'}, 'method': method, 'url': rt}
            cts = request2response(self._cli, http_request).text
        if not cts:
            try: cts = to_xml(rt)
            except: pass
        if cts:
            display_id, update = kwargs.pop('display_id', None), kwargs.pop('update', False)
            dhdl = display_id if isinstance(display_id, DisplayId) else DisplayId(display_id=display_id)
            if display_id and update: dhdl.update(cts)
            else: dhdl.display(cts)
            return None if dhdl is display_id else dhdl

    def _response(self, req:dict[str, Any]): return request2response(self._cli, req)
    
    def mount(self, prov:APIRouter|RouteProviderP, 
            path:str|None=None, name:str|None=None, index:str|None=None, 
            show:bool=True):
        ar = add_routes(self.app, prov, True, path, name)
        if hasattr(prov, '_mounted'): setattr(prov, '_mounted', True)
        if hasattr(prov, 'bridget'): setattr(prov, 'bridget', self)
        if show: self(index or f"{ar.to()}/")  # type: ignore
        return ar

    def swap(self,
            target, 
            content, 
            *, 
            # ---- swapSpec:SwapSpec, 
            swapStyle: SwapStyleT='innerHTML',
            swapDelay: int|None=None, settleDelay: int|None=None,
            transition: bool|None=None,
            # ignoreTitle: bool|None=None, head: Literal['merge', 'append']|None=None,
            scroll: str|None=None, scrollTarget: str|None=None,
            show: str|None=None, showTarget: str|None=None, focusScroll: bool|None=None,
            # ---- swapOptions=None,
            select: str|None=None, selectOOB: str|None=None,
            # eventInfo: dict|None=None,
            anchor: str|None=None,        
            # contextElement: str|None=None,
            # afterSwapCallback: Callable|None=None, afterSettleCallback: Callable|None=None,
        ): ...


FC.patch_to(BridgeBase)(swap)


In [ ]:
_send_sc =  """
// debugger;
const msg = %s;
console.log(`new message: ${JSON.stringify(msg)}`);
const { cmd, args } = msg;
if (cmd in htmx) {
    try {
        htmx[cmd](...(Array.isArray(args) ? args : Object.values(args)));
    } catch (e) {
        console.error(e);
    }
} else {
    console.warn(`Unknown HTMX command: ${cmd}`);
}
"""


class Bridge(BridgeBase):
    def send(self, msg:dict[str, Any]):
        json_msg = json.dumps(msg)
        display(Javascript(_send_sc % json_msg))


In [ ]:
def get_app(setup_scripts=False, app=None, appkw:dict[str, Any]={}, **cfargs) -> tuple[FastHTML, Bridge, MethodType]: 
    bridge_cfg.update(**cfargs)
    if setup_scripts: bridget_scripts(True)
    app = app or nb_app(**appkw)
    return app, Bridge().setup(app), app.route  # type: ignore


`BridgeBase` is a helper class that provides:

1. **Display functionality**: Renders FastHTML objects and route responses in notebook cells
2. **Component mounting**: Helper for mounting route providers
3. **HTMX API**: Python interface for HTMX commands (documented in 16_htmx.ipynb)

It serves as a mixin class for `Bridget`, which provides the full HTMX integration.

The class handles different types of "bridgeable" content:
- FastHTML objects (FT)
- Objects with `__ft__` or `__html__` methods
- Route paths
- Request mappings

In [ ]:
app, bridge, rt = get_app()

bridge();

In [ ]:
@app.get("/")  # type: ignore
def home():
    return "<h2>Hello, World</h2>"

bridge();


In [ ]:
@rt("/hi")
def get():
    return 'Hi there'

http_request = {
    'upload': {},
    'headers': {
        'HX-Request': 'true',
        'HX-Current-URL': 'vscode-webview://1ql27b...er'
    },
    'headerNames': {'hx-request': 'HX-Request', 'hx-current-url': 'HX-Current-URL'},
    'status': 0,
    'method': 'GET',
    'url': '/hi',
    'async': True,
    'timeout': 0,
    'withCredentials': False,
    'body': None
}

response = bridge._response(http_request)
test_eq(response.status_code, 200)
test_eq(response.text, 'Hi there')

console.print_json(data=(json_resp := httpx_response_to_json(response)))
test_eq(json_resp['status'], 200)
test_eq(json_resp['data'], 'Hi there')
test_eq(json_resp['headers']['content-length'], '8')

response

{
  "headers": {
    "content-length": "8",
    "content-type": "text/html; charset=utf-8",
    "last-modified": "Wed, 22 Jan 2025 19:02:33 GMT",
    "cache-control": "no-store, no-cache, must-revalidate"
  },
  "status": 200,
  "statusText": "OK",
  "data": "Hi there",
  "xml": null,
  "finalUrl": "http://nb/hi"
}

In [ ]:
bridge(http_request);

In [ ]:
class Buttons:
    def __ft__(self):
        return (
            Button(garlic=True, hx_get='/test', hx_select='button[vampire]', hx_swap='afterend')(_n,
                Style(self._css_.format('hsl(264 80% 47%)', 'hsl(264 80% 60%)')),
                'garlic ', Span('🧄', cls='icon'),
            _n), _n,
            Button(vampire=True, hx_get='/test', hx_select='button[garlic]', hx_swap='afterend')(_n,
                Style(self._css_.format('hsl(150 80% 47%)', 'hsl(150 80% 60%)')), 
                'vampire ', Span('🧛', cls='icon'),
            _n), _n,
        )
    _css_ = '''
    me {{ margin: 4px; padding: 10px 30px; min-width: 80px; background: {0}; border-bottom: 0.5rem solid hsl(264 80% 20%); }}
    me {{ color: antiquewhite; font-size: 14pt; font-variant: all-small-caps; font-weight: bold; }}
    me:hover {{ background: {1}; }}
    me span.icon {{ font-size:16pt; }}
'''


In [ ]:
@rt("/test")
def get():
    return Buttons()

http_request = {
    'headers': {'hx-request': '1'},
    'method': 'GET',
    'url': 'http://nb/test',
}

r = bridge._response(http_request)
display(Markdown(f"```HTML\n{r.text}\n```"))
r


```HTML
<button garlic hx-get="/test" hx-select="button[vampire]" hx-swap="afterend">
   <style>
    me { margin: 4px; padding: 10px 30px; min-width: 80px; background: hsl(264 80% 47%); border-bottom: 0.5rem solid hsl(264 80% 20%); }
    me { color: antiquewhite; font-size: 14pt; font-variant: all-small-caps; font-weight: bold; }
    me:hover { background: hsl(264 80% 60%); }
    me span.icon { font-size:16pt; }
</style>
garlic <span class="icon">🧄</span>
</button>
<button vampire hx-get="/test" hx-select="button[garlic]" hx-swap="afterend">
   <style>
    me { margin: 4px; padding: 10px 30px; min-width: 80px; background: hsl(150 80% 47%); border-bottom: 0.5rem solid hsl(264 80% 20%); }
    me { color: antiquewhite; font-size: 14pt; font-variant: all-small-caps; font-weight: bold; }
    me:hover { background: hsl(150 80% 60%); }
    me span.icon { font-size:16pt; }
</style>
vampire <span class="icon">🧛</span>
</button>

```

In [ ]:
bridge('/test');

In [ ]:
bridge(Buttons());

In [ ]:
# css = Style(':root {--pico-font-size:90%,--pico-font-family: Pacifico, cursive;}')

@app.route("/page")
def get():
    return (Title("Hello World"), 
            Main(H1('Hello, World'), cls="container"))

r = bridge._response(req := {
    # 'headers': {'hx-request': '1'},
    'method': 'GET',
    'url': 'http://nb/page',
})
display(Markdown(f"```HTML\n{r.text}\n```"))
bridge('page');


```HTML
 <!doctype html>
 <html>
   <head>
     <title>Hello World</title>
<script>
    function sendmsg() {
        window.parent.postMessage({height: document.documentElement.offsetHeight}, '*');
    }
    window.onload = function() {
        sendmsg();
        document.body.addEventListener('htmx:afterSettle',    sendmsg);
        document.body.addEventListener('htmx:wsAfterMessage', sendmsg);
    };</script>   </head>
   <body>
<main class="container">       <h1>Hello, World</h1>
</main>   </body>
 </html>

```

In [ ]:
dtl = Details(cls='pale', open=True)(
    Style('me details { border: 1px solid #aaa; padding: 0.5em 0.5em 0; } me summary { font-weight: bold; margin: -0.5em -0.5em 0; padding: 0.5em; } me pre { margin: 0; }'),
    Summary('What Lucy get?'),
    Div(cls='contents', style='display: flex; flex-direction: column;')(
        Pre('She'),
        Pre('got'),
        Pre('diamonds!')
    )
)
display(dtl)
bridge(dtl);


```html
<details open class="pale">  <style>me details { border: 1px solid #aaa; padding: 0.5em 0.5em 0; } me summary { font-weight: bold; margin: -0.5em -0.5em 0; padding: 0.5em; } me pre { margin: 0; }</style>
<summary>What Lucy get?</summary>  <div class="contents" style="display: flex; flex-direction: column;">
<pre>She</pre><pre>got</pre><pre>diamonds!</pre>  </div>
</details>
```

In [ ]:
def details_ft(
        *contents, 
        summary:str|None=None, 
        closed:bool=False, 
        direction:str='column', 
        height:str|None=None,
        contents_style:str='', item_style:str=''):
    style = f"display: flex; flex-direction: {direction};{height or ''}{contents_style or ''}"
    return Details(cls='pale', open=not closed)(
        Summary(summary),
        Div(cls='contents', style=style)(
            *(Div(style=item_style)(_) for _ in contents)
        )
    )

dtl = details_ft(
        *(Pre(_) for _ in ('She', 'got', 'diamonds')), 
        summary='What Lucy get?')
display(dtl)
bridge(dtl);


```html
<details open class="pale"><summary>What Lucy get?</summary>  <div class="contents" style="display: flex; flex-direction: column;">
    <div>
<pre>She</pre>    </div>
    <div>
<pre>got</pre>    </div>
    <div>
<pre>diamonds</pre>    </div>
  </div>
</details>
```

In [ ]:
%%HTML

<div id="output-99">Original</div>


In [ ]:
bridge.swap('#output-99', '<div>Swapped!</div>', swapStyle='innerHTML')


<IPython.core.display.Javascript object>

We can also use HTMX API through Bridge (documented in [21_htmx.ipynb](21_htmx.ipynb)).  

But we'll see a more convenient way with Bridget, without peppering the notebook of IPython Javascript display objects.


In [ ]:
did = bridge(Div('Hey, Foo!'), display_id='ultra-cool-display-id')

In [ ]:
bridge(Div('Hey, Bar!'), display_id=did, update=True)


Use `display_id` and `update` kwargs to modify or update an existing output cell.

# Bridget
> A widget that connects HTMX with FastHTML in notebooks.


In [ ]:
#| export

observer_js = BUNDLE_PATH / 'observer.js'
commander_js = BUNDLE_PATH / 'commander.js'
bridget_js = BUNDLE_PATH / 'bridget.js'


class Bridget(anywidget.AnyWidget, BridgeBase):
    "Bridge this notebook kernel and front-end, intercepting HTMX Ajax requests."
    _esm = anysource('debugger;', observer_js, commander_js, bridget_js, '''
export default async () => {
    const bridget = new Bridget();
    return {
        initialize: async (context) => await bridget.initialize(context)//,
        // render: (context) => bridget.render(context),
    }
}
''')

    request = T.Dict({}).tag(sync=True)   # Incoming HTMX requests
    response = T.Dict({}).tag(sync=True)  # Outgoing responses

    htmx = T.Bool(True).tag(sync=True)
    output_sels = T.List(['.output', '.jp-Cell-outputArea']).tag(sync=True)

    libraries = T.Dict({k:v.src for k,v in bridget_scripts().items()}).tag(sync=True)

    def __new__(cls, *args, **kwargs) -> Bridget:
        "Ensure single instance per notebook"
        if '__instance__' not in cls.__dict__: cls.__instance__ = super().__new__(cls, *args, **kwargs)
        return cls.__instance__
    
    def __init__(self, app=None, *args, logger:BasicLogger|None=None, show:bool=False, **kwargs):
        "Initialize with FastHTML app and optional debug display"
        if not hasattr(self, 'app'):
            assert app, "Bridget must be initialized with an app"
            self._loading = True
            self.setup(app)
            self.on_msg(self._on_message)
            self.logger = logger or BasicLogger().setup(height=400)
            self.logger.show('Loading Bridget...')
            super().__init__(*args, **kwargs)
        elif not self._loading and show:
            self.show()
        # if display: self.display()

    def show(self):
        "Display debug area"
        self.logger.close()
        self.logger = BasicLogger().setup(height=400)
        self.logger.show('Bridget already initialized!')
        self.logger.show(str(vars(bridge_cfg)))
    # def display(self):
    #     "Display widget and initialize debug output"
    #     from IPython.display import display
    #     if self.dhdl:
    #         self.dhdl.update('')
    #     display(self)
    #     self.dhdl = DisplayId()
    #     self.dhdl.display()
    
    def close(self):
        "Clean up widget instance and display"
        if hasattr(self, '__instance__'):
            del Bridget.__instance__
            self.logger.close()
        super().close()

    def _on_message(self, _, content, buffers):
        "Handle messages from the front-end"
        self._loading = False
        kind = content.get('kind')
        self.logger.show(f"_handle_message: {kind}")
        if kind == 'info':
            if content['info'] == 'initialized':
                self.logger.show('Bridget initialized!')
                self.logger.show(str(vars(bridge_cfg)))

    @T.observe('request')
    def _on_request(self, chg):
        "Handle incoming HTMX requests"
        req = chg['new']
        response = self._response(req)
        resp = httpx_response_to_json(response)
        resp['req_id'] = req['req_id']
        if bridge_cfg.debug_req: self._show_req(req, resp)
        self.response = resp
    
    def _show_req(self, req:dict[str, Any], resp:dict[str, Any]):
        self.logger.show(
                # pretty_repr(req, text=False) + pretty_repr(resp, text=False)
                to_xml(Div(cls='bridget-debug')(
                    'Request: ', NotStr(pretty_repr(req, text=False)),
                    'Response: ', NotStr(pretty_repr(resp, text=False))
                )),
                True
            )


def get_bridget(app=None, *args, logger:BasicLogger|None=None, show:bool=False, **kwargs):
    if Bridget.__instance__: 
        if show: Bridget.__instance__.show()
        return Bridget.__instance__
    return Bridget(app, *args, logger=logger, **kwargs)



Bridget is a specialized widget that:
1. Inherits from `BridgeBase` for FastHTML/display functionality
2. Uses [AnyWidget](https://anywidget.dev/) tooling-free, general widget solution, for browser-kernel communication
3. Implements a singleton pattern to ensure one instance per notebook

Attributes:
- `_esm`: Path to bundled JavaScript module
- `request`/`response`: Traitlets for HTMX communication
- `htmx`/`htmx_sels`: Configuration for HTMX integration

The core part is using the widget's bidirectional communication to replace HTMX's HTTP transport:
1. Browser: HTMX makes requests thinking it's talking to a server
2. Widget: Captures these requests via traitlets
3. Kernel: Processes requests using FastHTML routing
4. Widget: Returns responses that HTMX understands


## Bridget JavaScript Implementation [bridget.js](bridget.js)

Bridget's JavaScript code provides the browser-side implementation that:
1. Intercepts HTMX AJAX requests (SSE, WS in the future)
2. Routes them through the widget's communication channel
3. Processes responses back to HTMX
4. Manages HTMX initialization in notebook output cells

### Some details

#### Request Flow
1. HTMX makes an AJAX request
2. [xhook](https://github.com/jpillora/xhook/tree/main) intercepts it via `on_request`
3. Request is serialized and sent to kernel via traitlets
4. Kernel processes request and sends response
5. Response is processed by `response_changed`
6. HTMX receives response and updates DOM

#### HTMX Integration
1. `BridgetObserver` watches for new notebook output cells
2. New cells are processed with `htmx.process()`
3. HTMX attributes become active
4. AJAX requests are intercepted and routed through widget


Note that Bridget JS is backend agnostic. We're using FastHTML here because its the best for my quest of replacing ipywidgets as the main form of interactivity in a notebook, but it could be any other backend libarary.


## get_app
> Helper function to initialize the root-level app, bridget, and route.

In [ ]:
#| export

def get_app(setup_scripts=False, app=None, appkw:dict[str, Any]={}, **cfargs) -> tuple[FastHTML, Bridget, MethodType]: 
    bridge_cfg.update(**cfargs)
    if setup_scripts: bridget_scripts(True, htmx=False)
    app = app or nb_app(**appkw)
    return app, get_bridget(app), app.route  # type: ignore


In [ ]:
# cleanupwidgets('brt')
# try: del brt  # type: ignore
# except Exception: pass
# test_eq(getattr(Bridget, '__instance__', None), None)


In [ ]:
app, brt, rt = get_app()
test_is(brt, get_bridget())
test_is(brt.app, app)


`get_app` setup the root app and bridget, and returns the route decorator..

In [ ]:
get_bridget(show=True)


Bridget(libraries={'htmx': 'https://unpkg.com/htmx.org@next/dist/htmx.js', 'fasthtml_js': 'https://cdn.jsdeliv…

Once loaded, this only show widget feedback and debug area. The widget itself is headless and already loaded above.

In [ ]:
# brt.close()
# del brt

Let's see Bridget in action. (Look above to see the request and response)

In [ ]:
bridge_cfg.debug_req = True
bridge_cfg.auto_show = True


In [ ]:
def counter():
    n = 0
    @rt('/inc')
    def increment():
        nonlocal n
        n += 1
        return f"{n}"
    return Div()(
        Button(hx_get='/inc', hx_target='find span', hx_swap='textContent')(
            'Count: ', Span(f"{n}")))

counter()


div((button(('Count: ', span(('0',),{})),{'hx-get': '/inc', 'hx-swap': 'textContent', 'hx-target': 'find span'}),),{})

In [ ]:
def random_hsl(saturation=50, lightness=90):
    hue = random.randint(0, 360)
    return f"hsl({hue} {saturation}% {lightness}%)"

def counter(n=0):
    @rt('/inc')
    def increment(n:int):
        return Button(f"Count: {n+1}", value=f"{n+1}", name='n', 
            hx_post='/inc', hx_swap='outerHTML', 
            style=f"background-color:{random_hsl()}; font-weight: bold")
    return increment(n-1)

counter()


button(('Count: 0',),{'value': '0', 'name': 'n', 'hx-post': '/inc', 'hx-swap': 'outerHTML', 'style': 'background-color:hsl(191 50% 90%); font-weight: bold'})

In [ ]:
@rt("/test")
def get():
    return Buttons()


html = Div()(
    H2('HTMX Test'),
    Div('Swapped DOM elements are styled instantly when they arrive.'),
    Buttons(),
)

# brt(html); # or simply, if bridge_cfg.auto_show is True:
html


div((h2(('HTMX Test',),{}), div(('Swapped DOM elements are styled instantly when they arrive.',),{}), <__main__.Buttons object>),{})

An example from [gnat](https://github.com/gnat)'s [css-scope-inline](https://github.com/gnat/css-scope-inline). Click the buttons.

See [details_json.ipynb](30_details_json.ipynb) for a lazy JSON browser.

:::{.callout-tip}
## Why are my output cells un-styled and/or inactive?
Because all changes made with Bridge/Bridget/FastHTML/HTMX are **transient**.
:::

Let's talk about what's happening under the hood:

We're modifying the DOM - the HTML structure in your browser - at runtime. This is **not** a notebook editor; it's a runtime tool. For anything to work, you need to execute the cells.

Here's the thing: a notebook is just **JSON**. Ultimately, output cells are just HTML (derived from any IPython displayable object), environments like VSCode won't render them until display time.

When you open a notebook, tjhe front-end usually takes a lazy approach (VSCode/Cursor more than others):
- It renders existing outputs from the JSON (including JavaScript)
- But it won't execute any cells automatically
- Even ipywidgets get special (and sometimes quirky) treatment

Think of the notebook's JSON as a snapshot from when you ran the cells. Any DOM changes we make don't get saved back to this JSON.

This means:
1. Bridge needs explicit initialization to load its JS/CSS
2. On load, saved notebooks will execute the JavaScript/HTML put there by cell runs
3. Kernel-side code won't run until you execute the cells

Could we make Bridget modify the actual notebook outputs? Technically, yes, maybe. But given the labyrinthine complexity of the Jupyter ecosystem (and the mountains of JavaScript involved), I get a headache just thinking about it. Some mountains are better left unclimbed! 😅

# Simple widget


In [ ]:
bridge_cfg.auto_show = True
bridge_cfg.auto_mount = True
bridge_cfg.debug_req = True


In [ ]:

class BWidget(T.HasTraits, RouteProvider):
    bridget: Bridget = None  # type: ignore
    _mounted = False
    def __ft__(self): ...
    def _ipython_display_(self):
        brt = get_bridget()
        if bridge_cfg.auto_mount and not self._mounted: brt.mount(self, show=False)
        brt(self);

class BValue(BWidget):
    value=T.CInt(0).tag(sync=True)
    _updating = False
    
    @contextmanager
    def _update_ctx(self):
        self._updating = True
        yield
        self._updating = False

    @T.observe('value')
    def on_value(self, _):
        if self.bridget and not self._updating:
            self.bridget.swap(f"#{self.ar.name()}", to_xml(self.__ft__()), swapStyle='innerHTML')

    @ar.post('/value')
    def changed(self, value:int):
        with self._update_ctx(): self.value = value
        return str(value)


@dataclasses.dataclass
class BIntSlider(BValue):
    min:int=0
    max:int=100
    step:int=1
    readout:bool=True
    readout_format:str='d'
    
    def __ft__(self):
        if bridge_cfg.auto_mount and not self._mounted: get_bridget().mount(self, show=False)
        return Div(id=self.ar.name(), cls='bridget slider')(
            Label(_for='value')('Scale'), _n,
            Input(type='range', name='value', min=self.min, max=self.max, step=self.step, value=self.value,
                # hx_post=f"{self.ar.to()}{self.ar.to('changed')}", hx_trigger='input changed', 
                hx_post=self.ar.to('changed'), hx_trigger='input changed', 
                hx_target='next text', hx_swap='textContent'),
            Text(id='spanscale', style='inline')(self.value), _n
        )

app.routes.clear()

rp = BIntSlider()
display(rp)
get_bridget(show=True)


Bridget(libraries={'htmx': 'https://unpkg.com/htmx.org@next/dist/htmx.js', 'fasthtml_js': 'https://cdn.jsdeliv…

In [ ]:
rp.value = 77

In [ ]:
sld2 = BIntSlider(step=2)
sld2

In [ ]:
sld3 = BIntSlider()
sld4 = BIntSlider()
# brt.mount(sld3, show=False)
# brt.mount(sld4, show=False)
T.link((sld3, 'value'), (sld4, 'value'))

box = Div(style='display: flex; gap: 1em;')(sld3, sld4)
box


div((BIntSlider(min=0, max=100, step=1, readout=True, readout_format='d'), BIntSlider(min=0, max=100, step=1, readout=True, readout_format='d')),{'style': 'display: flex; gap: 1em;'})

In [ ]:
sld3.value = 22
test_eq(sld4.value, 22)


# Hydrate (TBD)
> Can we edit `.ipynb`s directly to capture actual output without `nbformat` or editing the JSON in disk?


In [ ]:
def hydrate(bridget=True, app: FastHTML | None=None, appkw:dict[str, Any]={}, **kwargs): 
    app, bridge, rt = get_app(True, app, appkw=appkw, **kwargs)
    bridget = Bridget(bridge) if bridget else None
    return app, bridge, rt, bridget

# hydrate()


# What about WebSockets and SSE Support?



While HTMX supports WebSockets and Server-Sent Events (SSE) through extensions([1](https://htmx.org/docs/#web-sockets-sse)), this proof-of-concept focuses only on AJAX functionality for several reasons:

1. **Core Functionality**: HTMX is primarily an AJAX framework - WS and SSE support are add-ons with simpler implementations([2](https://htmx.org/docs/#extensions))

2. **Proof of Concept**: For demonstrating the viability of using HTMX in notebooks, AJAX support is sufficient

3. **Future Extension**: Adding WS/SSE support would be straightforward since:
   - The notebook Comm layer already uses WebSockets
   - HTMX's extension system is well-documented([3](https://htmx.org/docs/#creating-extensions))
   - The transport layer replacement pattern is already established with AJAX


# Colophon
----


In [ ]:
import inspect
import shutil
from pathlib import Path

import bridget
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [ ]:
if FC.IN_NOTEBOOK:
    nb_path = '32_bridget.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)

    for fn in ('bridget.js', ):
        shutil.copyfile(fn, Path(inspect.getfile(bridget)).parent/fn);
